In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import psycopg2

from datetime import date


In [2]:
# Function to Get the current
# working directory
def current_path():
    print("Current working directory before")
    print(os.getcwd())
    print()
 
 
# Driver's code
# Printing CWD before
current_path()
 
# Changing the CWD
os.chdir('../../../Work Order - Data/WO_Folders/WO_2022-2023')
 
# Printing CWD after
current_path()

Current working directory before
C:\Users\E178162\Documents\Work Orders\DB\Jupyter Notebooks\Data Cleanse

Current working directory before
C:\Users\E178162\Documents\Work Orders\Work Order - Data\WO_Folders\WO_2022-2023



In [3]:
cwd = os.getcwd()  # Get the current working directory (cwd)
file = os.listdir(cwd)[0]  # Get all the files in that directory
print("Files in %r: %s" % (cwd, file))

Files in 'C:\\Users\\E178162\\Documents\\Work Orders\\Work Order - Data\\WO_Folders\\WO_2022-2023': Work_Order_(01_23_2023_03_12_57_PM).csv


In [4]:
os.path.exists(file)

True

In [5]:
df = pd.read_csv(file)

In [6]:
df.shape

(31100, 27)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31100 entries, 0 to 31099
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Work Order #           31100 non-null  int64  
 1   WO Status              31100 non-null  object 
 2   Priority               29484 non-null  object 
 3   Work Category          31094 non-null  object 
 4   Problem                31094 non-null  object 
 5   Source Site            31100 non-null  object 
 6   Source Location        30034 non-null  object 
 7   Area/Room #            28313 non-null  object 
 8   Completed              27098 non-null  object 
 9   Created On             31100 non-null  object 
 10  Originated             31100 non-null  object 
 11  Originator             31100 non-null  object 
 12  Originator First Name  30423 non-null  object 
 13  Source User            30607 non-null  object 
 14  Work requested         31099 non-null  object 
 15  Ca

In [8]:
df['Date Open'] = pd.to_datetime(df['Created On']).dt.date
df['Date Closed'] = pd.to_datetime(df['Completed']).dt.date #Creates new column and sets data type from DateTime -> Date

In [9]:
df['Date Difference'] = date.today() - df['Date Open'] #Takes the difference between today's date from Date Open
df['Date Difference'] = df['Date Difference'].dt.days #Converts DateTime Type to Integer

In [10]:
df['Duration'] = df['Date Closed'] - df['Date Open'] #Takes the diffrence between Date Closed from Date Open
df['Duration'] = df['Duration'].dt.days #Converts DateTime Type to Integer
df['Duration'] = df['Duration'].abs() #Gets the Absolute Value of Column Values

In [11]:
df['Site_ID'] = df['Source Site'].str[-3:] #Takes the last 3 characters froms string in this case, we get extract the Site Code

In [12]:
df['School Year'] = '2022 - 2023'

In [13]:
df['Duration'] = df['Duration'].fillna(df['Date Difference']) #Fill null values from Duration column with values from Date Difference

In [14]:
df['Row Count'] = 1

In [15]:
df = df.drop(columns=['Area/Room #', 'Originator', 'Originator First Name', 'Work requested', 'Cause', 'Action Taken', 'Comments' ,'Longitude', 'Latitude', 'PM Title', 'Completed','Created On', 'Originated']) #Drops/Removes the columns from table

In [16]:
df = df[df['Site_ID'].str.contains("ZZ-|032|941") == False]

In [17]:
df.head()

,Work Order #,WO Status,Priority,Work Category,Problem,Source Site,Source Location,Source User,Work Type,Labor Hrs,...,Labor Cost,Total Hrs,Grand Total,Date Open,Date Closed,Date Difference,Duration,Site_ID,School Year,Row Count
0,260769,New Request,Low,HVAC - Air Conditioning,Not Heating,Bowie HS - 013,Bowie B213,Edward Botello,NaN,0.0,...,0.0,0.0,0.0,2023-01-23,NaT,0,0.0,013,2022 - 2023,1
1,260768,New Request,Low,Electrical,Other,Eastside HS - 019,Eastside A116,Thorsten Smaglinski,NaN,0.0,...,0.0,0.0,0.0,2023-01-23,NaT,0,0.0,019,2022 - 2023,1
2,260767,In Progress,High,AV/Electronics,Document Camera,Langford ES - 168,Langford 705,Al Gonzalez A/V,Reactive Maintenance,0.0,...,0.0,0.0,0.0,2023-01-23,NaT,0,0.0,168,2022 - 2023,1
3,260766,New Request,Low,Building Operations,Other,Covington MS - 057,FINE ARTS,Steven Harness,Safety & Security,0.0,...,0.0,0.0,0.0,2023-01-23,NaT,0,0.0,057,2022 - 2023,1
4,260765,New Request,Low,Locksmith,Locks,Menchaca ES - 147,Menchaca A119,Arthur Mendez,Safety & Security,0.0,...,0.0,0.0,0.0,2023-01-23,NaT,0,0.0,147,2022 - 2023,1


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31099 entries, 0 to 31099
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Work Order #     31099 non-null  int64  
 1   WO Status        31099 non-null  object 
 2   Priority         29483 non-null  object 
 3   Work Category    31093 non-null  object 
 4   Problem          31093 non-null  object 
 5   Source Site      31099 non-null  object 
 6   Source Location  30033 non-null  object 
 7   Source User      30606 non-null  object 
 8   Work Type        18018 non-null  object 
 9   Labor Hrs        31099 non-null  float64
 10  Part Cost        31099 non-null  float64
 11  Labor Cost       31099 non-null  float64
 12  Total Hrs        31099 non-null  float64
 13  Grand Total      31099 non-null  float64
 14  Date Open        31099 non-null  object 
 15  Date Closed      27097 non-null  object 
 16  Date Difference  31099 non-null  int64  
 17  Duration    

In [19]:
df = df.drop_duplicates(subset='Work Order #', keep='first')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31092 entries, 0 to 31099
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Work Order #     31092 non-null  int64  
 1   WO Status        31092 non-null  object 
 2   Priority         29476 non-null  object 
 3   Work Category    31086 non-null  object 
 4   Problem          31086 non-null  object 
 5   Source Site      31092 non-null  object 
 6   Source Location  30026 non-null  object 
 7   Source User      30599 non-null  object 
 8   Work Type        18012 non-null  object 
 9   Labor Hrs        31092 non-null  float64
 10  Part Cost        31092 non-null  float64
 11  Labor Cost       31092 non-null  float64
 12  Total Hrs        31092 non-null  float64
 13  Grand Total      31092 non-null  float64
 14  Date Open        31092 non-null  object 
 15  Date Closed      27092 non-null  object 
 16  Date Difference  31092 non-null  int64  
 17  Duration    

In [20]:
clean_tbl_name = file.lower().replace(" ","").replace("?", "")\
                    .replace("-", "_").replace(r"/","_").replace("\\", "_")\
                    .replace("%", "").replace(")", "").replace(r"(","").replace("$", "")\
                    .replace("pm", "").replace("1", "").replace("2", "").replace("3", "")\
                    .replace("4", "").replace("5", "").replace("6", "").replace("7", "")\
                    .replace("8", "").replace("9", "").replace("0", "").replace("_", "")\
                    .replace("workorder", "updated_work_orders")
clean_tbl_name

'updated_work_orders.csv'

In [21]:
df.columns = [x.lower().replace(" ","_").replace("?", "").replace("#", "id")\
                    .replace("-", "_").replace(r"/","_").replace("\\", "_")\
                    .replace("%", "").replace(")", "").replace(r"(","").replace("$", "") for x in df.columns]

df.columns

Index(['work_order_id', 'wo_status', 'priority', 'work_category', 'problem',
       'source_site', 'source_location', 'source_user', 'work_type',
       'labor_hrs', 'part_cost', 'labor_cost', 'total_hrs', 'grand_total',
       'date_open', 'date_closed', 'date_difference', 'duration', 'site_id',
       'school_year', 'row_count'],
      dtype='object')

In [22]:
df.dtypes

work_order_id        int64
wo_status           object
priority            object
work_category       object
problem             object
source_site         object
source_location     object
source_user         object
work_type           object
labor_hrs          float64
part_cost          float64
labor_cost         float64
total_hrs          float64
grand_total        float64
date_open           object
date_closed         object
date_difference      int64
duration           float64
site_id             object
school_year         object
row_count            int64
dtype: object

In [23]:
replacements = {
    'object': 'varchar',
    'float64': 'float',
    'int64': 'int',
    'datetime64': 'timestamp',
    'timedelta64[ns]': 'varchar'
}

replacements

{'object': 'varchar',
 'float64': 'float',
 'int64': 'int',
 'datetime64': 'timestamp',
 'timedelta64[ns]': 'varchar'}

In [24]:
col_str = ", ".join("{} {}".format(n, d) for (n,d) in zip(df.columns, df.dtypes.replace(replacements)))
col_str

'work_order_id int, wo_status varchar, priority varchar, work_category varchar, problem varchar, source_site varchar, source_location varchar, source_user varchar, work_type varchar, labor_hrs float, part_cost float, labor_cost float, total_hrs float, grand_total float, date_open varchar, date_closed varchar, date_difference int, duration float, site_id varchar, school_year varchar, row_count int'

In [25]:
conn_string = "host=localhost\
               dbname='work_orders'\
               user=postgres password='postgres'"

conn = psycopg2.connect(conn_string)
cursor = conn.cursor()
print('opened database successfully!')

opened database successfully!


In [26]:
#drop table
cursor.execute("drop table if exists work_orders;")

In [27]:
#create table
cursor.execute("CREATE TABLE work_orders \
(work_order_id INT NOT NULL PRIMARY KEY, wo_status VARCHAR(50), priority VARCHAR(50), \
work_category VARCHAR(50) REFERENCES work_category(work_category), problem VARCHAR(50), \
source_site VARCHAR(50), source_location VARCHAR(50), source_user VARCHAR(50), work_type VARCHAR(50),\
labor_hrs float, part_cost float, labor_cost float, total_hrs float, grand_total float, \
date_open DATE, date_closed DATE, \
date_difference float, duration float, site_id INT REFERENCES schools(site_id), school_year VARCHAR(25), row_count INT NOT NULL)")

In [28]:
df.to_csv('updated_work_orders.csv', header=df.columns, index=False, encoding='utf=8')

my_file = open('updated_work_orders.csv')
print('File opened in memory and export successful!')

File opened in memory and export successful!


In [29]:
#upload to db

SQL_STATEMENT = """
COPY work_orders FROM STDIN WITH 
    CSV
    HEADER
    DELIMITER AS ','
"""

cursor.copy_expert(sql = SQL_STATEMENT, file=my_file)
print('file copied to db!')

file copied to db!


cursor.execute("grant select on table work_orders to public")
conn.commit()

cursor.close()
print('CSV imported to DB successfully!')